In [1]:
import numpy as np
import pandas as pd
import os
import json
import re
import string
import nltk

In [2]:
TRAIN_DATA_PATH = "C:/Users/user/Desktop/bilm-tf-master/textdataset/IMDB/aclImdb/train"
TEST_DATA_PATH = "C:/Users/user/Desktop/bilm-tf-master/textdataset/IMDB/aclImdb/test"

def read_text_file(path):
    labels = ['neg','pos']
    if os.path.exists(path):
        text=[]
        text_label =[]
        for directory_name in os.listdir(path):
            if directory_name in labels:
                label_index = labels.index(directory_name)
                data_path = os.path.join(path,directory_name)
                for file in os.listdir(data_path):
                    with open(os.path.join(data_path,file),'r', encoding='utf-8') as f:
                        text.append(f.read())
                        text_label.append(label_index)
        return pd.DataFrame(text,columns =['texts']),pd.DataFrame(text_label,columns =['label'])
    
x_train,y_train = read_text_file(TRAIN_DATA_PATH) 
x_test,y_test = read_text_file(TEST_DATA_PATH) 

In [3]:
train = pd.concat([x_train, y_train], axis=1)
test = pd.concat([x_test, y_test], axis=1)

train.drop_duplicates(inplace=True)
test.drop_duplicates(inplace=True)

In [4]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

train[['texts']] = train[['texts']].applymap(lambda x:remove_punctuation(x))
train[['texts']] = train[['texts']].applymap(lambda x:x.lower())
test[['texts']] = test[['texts']].applymap(lambda x:remove_punctuation(x))
test[['texts']] = test[['texts']].applymap(lambda x:x.lower())

In [5]:
def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations
    #text = re.sub('\n', '', text)
    #text = re.sub('\w*\d\w*', '', text)
    return text

In [6]:
X_train = train.drop(columns=['label'])
X_test = test.drop(columns=['label'])
y_train = train['label']
y_test = test['label']

X_train = X_train['texts'].apply(clean_text)
X_test = X_test['texts'].apply(clean_text)

pattern = '[^a-z ]'
Clean_X_train=[]
Clean_X_test=[]

for sen in X_train:
    Clean_X_train.append(re.sub(pattern, '', str(sen)))
    
for sen in X_test:
    Clean_X_test.append(re.sub(pattern, '', str(sen)))

In [7]:
y_train=list(y_train)
y_test=list(y_test)

In [8]:
train_df = pd.DataFrame({'X_train': Clean_X_train, 'y_train': y_train})
test_df = pd.DataFrame({'X_test': Clean_X_test, 'y_test': y_test})

In [9]:
# 레이블 값에 따라 데이터프레임을 그룹화하고 각 그룹에서 8000개의 샘플을 랜덤하게 추출
train_df = train_df.groupby('y_train').apply(lambda x: x.sample(n=8000, random_state=42))

# 레이블 값에 따라 데이터프레임을 그룹화하고 각 그룹에서 2000개의 샘플을 랜덤하게 추출
test_df = test_df.groupby('y_test').apply(lambda x: x.sample(n=2000, random_state=42))

# 인덱스를 재설정합니다. drop=True 옵션을 사용하여 기존 인덱스를 제거합니다.
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [10]:
x_train = train_df['X_train'].tolist()
y_train = train_df['y_train'].tolist()
x_test = test_df['X_test'].tolist()
y_test = test_df['y_test'].tolist()

In [11]:
to_txt=x_train+x_test
y=y_train+y_test

엘모벡터 생성

In [12]:
import os
os.chdir('C:\\Users\\user\\Desktop\\bilm-tf-master')

In [13]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers
 
#사전 학습된 모델이 있는 디렉토리
datadir = os.path.join('swb', 'model(IMDB256)') 
vocab_file = os.path.join(datadir, 'vocab.txt')
options_file = os.path.join(datadir, 'options.json')
weight_file = os.path.join(datadir, 'swb_weights.hdf5')
 
batcher = Batcher(vocab_file, 50)
 
context_character_ids = tf.placeholder('int32', shape=(None, None, 50))
 
bilm = BidirectionalLanguageModel(options_file, weight_file)
 
context_embeddings_op = bilm(context_character_ids)
 
elmo_context_input = weight_layers('input', context_embeddings_op, l2_coef=0.0)

#테스트 임베딩
raw_context = ['Technology has advanced so much in new scientific world',
                'My child participated in fancy dress competition',
                'Fashion industry has seen tremendous growth in new designs']
 
tokenized_context = [sentence.split() for sentence in raw_context]
print(tokenized_context)

C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework

USING SKIP CONNECTIONS
[['Technology', 'has', 'advanced', 'so', 'much', 'in', 'new', 'scientific', 'world'], ['My', 'child', 'participated', 'in', 'fancy', 'dress', 'competition'], ['Fashion', 'industry', 'has', 'seen', 'tremendous', 'growth', 'in', 'new', 'designs']]


In [14]:
with tf.Session() as sess:    
    sess.run(tf.global_variables_initializer())     
    context_ids = batcher.batch_sentences(tokenized_context)
    print("Shape of context ids = ", context_ids.shape)
    elmo_context_input_ = sess.run(
        elmo_context_input['weighted_op'],
        feed_dict={context_character_ids: context_ids}
    )
 
print("Shape of generated embeddings = ",elmo_context_input_.shape)

Shape of context ids =  (3, 11, 50)
Shape of generated embeddings =  (3, 9, 256)


In [15]:
euc_dist_bet_tech_computer = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[0,0,:])
euc_dist_bet_computer_fashion = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[2,0,:])
cos_dist_bet_tech_computer = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[0,0,:])
cos_dist_bet_computer_fashion = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[2,0,:])
 
print("Euclidean Distance Comparison - ")
print("\nDress-Technology = ",np.round(euc_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(euc_dist_bet_computer_fashion,2))
print("\n\nCosine Distance Comparison - ")
print("\nDress-Technology = ",np.round(cos_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(cos_dist_bet_computer_fashion,2))

Euclidean Distance Comparison - 

Dress-Technology =  11.98 
Dress-Fashion =  11.92


Cosine Distance Comparison - 

Dress-Technology =  0.74 
Dress-Fashion =  0.73


임베딩

In [16]:
#메모리 오류로 임베딩 되지 않는 문서의 인덱스를 찾기위한 리스트
except_lst=[]

In [17]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers

for i in range(len(x_train)):
    try:
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            context_ids = batcher.batch_sentences([x_train[i].split()])
            elmo_context_input_ = sess.run(elmo_context_input['weighted_op'],feed_dict={context_character_ids: context_ids}) 
        print("Shape of generated embeddings = ",elmo_context_input_.shape)
        #임베딩된 문서 임베딩 행렬을 저장
        np.save('C:/Users/user/Desktop/Multimodal TextCuboid/IMDB 분류/elmo_embedding/train(IMDB256)/doc%d'%i,elmo_context_input_[0])
    except:
        except_lst.append(i)

Shape of generated embeddings =  (1, 55, 256)
Shape of generated embeddings =  (1, 138, 256)
Shape of generated embeddings =  (1, 649, 256)
Shape of generated embeddings =  (1, 59, 256)
Shape of generated embeddings =  (1, 180, 256)
Shape of generated embeddings =  (1, 687, 256)
Shape of generated embeddings =  (1, 236, 256)
Shape of generated embeddings =  (1, 459, 256)
Shape of generated embeddings =  (1, 87, 256)
Shape of generated embeddings =  (1, 373, 256)
Shape of generated embeddings =  (1, 215, 256)
Shape of generated embeddings =  (1, 439, 256)
Shape of generated embeddings =  (1, 206, 256)
Shape of generated embeddings =  (1, 232, 256)
Shape of generated embeddings =  (1, 158, 256)
Shape of generated embeddings =  (1, 169, 256)
Shape of generated embeddings =  (1, 55, 256)
Shape of generated embeddings =  (1, 219, 256)
Shape of generated embeddings =  (1, 263, 256)
Shape of generated embeddings =  (1, 453, 256)
Shape of generated embeddings =  (1, 79, 256)
Shape of generated

Shape of generated embeddings =  (1, 237, 256)
Shape of generated embeddings =  (1, 262, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 172, 256)
Shape of generated embeddings =  (1, 254, 256)
Shape of generated embeddings =  (1, 146, 256)
Shape of generated embeddings =  (1, 245, 256)
Shape of generated embeddings =  (1, 978, 256)
Shape of generated embeddings =  (1, 414, 256)
Shape of generated embeddings =  (1, 123, 256)
Shape of generated embeddings =  (1, 63, 256)
Shape of generated embeddings =  (1, 161, 256)
Shape of generated embeddings =  (1, 123, 256)
Shape of generated embeddings =  (1, 337, 256)
Shape of generated embeddings =  (1, 183, 256)
Shape of generated embeddings =  (1, 147, 256)
Shape of generated embeddings =  (1, 213, 256)
Shape of generated embeddings =  (1, 355, 256)
Shape of generated embeddings =  (1, 213, 256)
Shape of generated embeddings =  (1, 121, 256)
Shape of generated embeddings =  (1, 107, 256)
Shape of genera

Shape of generated embeddings =  (1, 87, 256)
Shape of generated embeddings =  (1, 242, 256)
Shape of generated embeddings =  (1, 167, 256)
Shape of generated embeddings =  (1, 356, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 141, 256)
Shape of generated embeddings =  (1, 438, 256)
Shape of generated embeddings =  (1, 115, 256)
Shape of generated embeddings =  (1, 208, 256)
Shape of generated embeddings =  (1, 120, 256)
Shape of generated embeddings =  (1, 325, 256)
Shape of generated embeddings =  (1, 83, 256)
Shape of generated embeddings =  (1, 237, 256)
Shape of generated embeddings =  (1, 112, 256)
Shape of generated embeddings =  (1, 123, 256)
Shape of generated embeddings =  (1, 500, 256)
Shape of generated embeddings =  (1, 76, 256)
Shape of generated embeddings =  (1, 113, 256)
Shape of generated embeddings =  (1, 510, 256)
Shape of generated embeddings =  (1, 96, 256)
Shape of generated embeddings =  (1, 112, 256)
Shape of generated

Shape of generated embeddings =  (1, 187, 256)
Shape of generated embeddings =  (1, 212, 256)
Shape of generated embeddings =  (1, 563, 256)
Shape of generated embeddings =  (1, 58, 256)
Shape of generated embeddings =  (1, 86, 256)
Shape of generated embeddings =  (1, 110, 256)
Shape of generated embeddings =  (1, 123, 256)
Shape of generated embeddings =  (1, 63, 256)
Shape of generated embeddings =  (1, 206, 256)
Shape of generated embeddings =  (1, 159, 256)
Shape of generated embeddings =  (1, 118, 256)
Shape of generated embeddings =  (1, 544, 256)
Shape of generated embeddings =  (1, 146, 256)
Shape of generated embeddings =  (1, 319, 256)
Shape of generated embeddings =  (1, 195, 256)
Shape of generated embeddings =  (1, 110, 256)
Shape of generated embeddings =  (1, 178, 256)
Shape of generated embeddings =  (1, 260, 256)
Shape of generated embeddings =  (1, 142, 256)
Shape of generated embeddings =  (1, 199, 256)
Shape of generated embeddings =  (1, 142, 256)
Shape of generat

Shape of generated embeddings =  (1, 121, 256)
Shape of generated embeddings =  (1, 170, 256)
Shape of generated embeddings =  (1, 90, 256)
Shape of generated embeddings =  (1, 175, 256)
Shape of generated embeddings =  (1, 133, 256)
Shape of generated embeddings =  (1, 120, 256)
Shape of generated embeddings =  (1, 110, 256)
Shape of generated embeddings =  (1, 197, 256)
Shape of generated embeddings =  (1, 179, 256)
Shape of generated embeddings =  (1, 110, 256)
Shape of generated embeddings =  (1, 124, 256)
Shape of generated embeddings =  (1, 199, 256)
Shape of generated embeddings =  (1, 211, 256)
Shape of generated embeddings =  (1, 226, 256)
Shape of generated embeddings =  (1, 952, 256)
Shape of generated embeddings =  (1, 184, 256)
Shape of generated embeddings =  (1, 126, 256)
Shape of generated embeddings =  (1, 166, 256)
Shape of generated embeddings =  (1, 117, 256)
Shape of generated embeddings =  (1, 210, 256)
Shape of generated embeddings =  (1, 114, 256)
Shape of gener

...

Shape of generated embeddings =  (1, 148, 256)
Shape of generated embeddings =  (1, 85, 256)
Shape of generated embeddings =  (1, 418, 256)
Shape of generated embeddings =  (1, 213, 256)
Shape of generated embeddings =  (1, 138, 256)
Shape of generated embeddings =  (1, 352, 256)
Shape of generated embeddings =  (1, 120, 256)
Shape of generated embeddings =  (1, 427, 256)
Shape of generated embeddings =  (1, 85, 256)
Shape of generated embeddings =  (1, 377, 256)
Shape of generated embeddings =  (1, 111, 256)
Shape of generated embeddings =  (1, 247, 256)
Shape of generated embeddings =  (1, 274, 256)
Shape of generated embeddings =  (1, 56, 256)
Shape of generated embeddings =  (1, 232, 256)
Shape of generated embeddings =  (1, 419, 256)
Shape of generated embeddings =  (1, 122, 256)
Shape of generated embeddings =  (1, 127, 256)
Shape of generated embeddings =  (1, 214, 256)
Shape of generated embeddings =  (1, 299, 256)
Shape of generated embeddings =  (1, 666, 256)
Shape of generat

Shape of generated embeddings =  (1, 100, 256)
Shape of generated embeddings =  (1, 63, 256)
Shape of generated embeddings =  (1, 75, 256)
Shape of generated embeddings =  (1, 421, 256)
Shape of generated embeddings =  (1, 132, 256)
Shape of generated embeddings =  (1, 117, 256)
Shape of generated embeddings =  (1, 297, 256)
Shape of generated embeddings =  (1, 208, 256)
Shape of generated embeddings =  (1, 138, 256)
Shape of generated embeddings =  (1, 105, 256)
Shape of generated embeddings =  (1, 124, 256)
Shape of generated embeddings =  (1, 129, 256)
Shape of generated embeddings =  (1, 93, 256)
Shape of generated embeddings =  (1, 202, 256)
Shape of generated embeddings =  (1, 119, 256)
Shape of generated embeddings =  (1, 275, 256)
Shape of generated embeddings =  (1, 197, 256)
Shape of generated embeddings =  (1, 209, 256)
Shape of generated embeddings =  (1, 264, 256)
Shape of generated embeddings =  (1, 182, 256)
Shape of generated embeddings =  (1, 122, 256)
Shape of generat

Shape of generated embeddings =  (1, 134, 256)
Shape of generated embeddings =  (1, 848, 256)
Shape of generated embeddings =  (1, 210, 256)
Shape of generated embeddings =  (1, 455, 256)
Shape of generated embeddings =  (1, 303, 256)
Shape of generated embeddings =  (1, 207, 256)
Shape of generated embeddings =  (1, 391, 256)
Shape of generated embeddings =  (1, 201, 256)
Shape of generated embeddings =  (1, 216, 256)
Shape of generated embeddings =  (1, 170, 256)
Shape of generated embeddings =  (1, 249, 256)
Shape of generated embeddings =  (1, 117, 256)
Shape of generated embeddings =  (1, 439, 256)
Shape of generated embeddings =  (1, 422, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 183, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 190, 256)
Shape of generated embeddings =  (1, 187, 256)
Shape of generated embeddings =  (1, 374, 256)
Shape of generated embeddings =  (1, 110, 256)
Shape of genera

Shape of generated embeddings =  (1, 65, 256)
Shape of generated embeddings =  (1, 150, 256)
Shape of generated embeddings =  (1, 123, 256)
Shape of generated embeddings =  (1, 65, 256)
Shape of generated embeddings =  (1, 359, 256)
Shape of generated embeddings =  (1, 187, 256)
Shape of generated embeddings =  (1, 125, 256)
Shape of generated embeddings =  (1, 125, 256)
Shape of generated embeddings =  (1, 133, 256)
Shape of generated embeddings =  (1, 223, 256)
Shape of generated embeddings =  (1, 297, 256)
Shape of generated embeddings =  (1, 132, 256)
Shape of generated embeddings =  (1, 127, 256)
Shape of generated embeddings =  (1, 1822, 256)
Shape of generated embeddings =  (1, 316, 256)
Shape of generated embeddings =  (1, 148, 256)
Shape of generated embeddings =  (1, 206, 256)
Shape of generated embeddings =  (1, 234, 256)
Shape of generated embeddings =  (1, 132, 256)
Shape of generated embeddings =  (1, 204, 256)
Shape of generated embeddings =  (1, 152, 256)
Shape of gener

Shape of generated embeddings =  (1, 134, 256)
Shape of generated embeddings =  (1, 116, 256)
Shape of generated embeddings =  (1, 159, 256)
Shape of generated embeddings =  (1, 116, 256)
Shape of generated embeddings =  (1, 180, 256)
Shape of generated embeddings =  (1, 118, 256)
Shape of generated embeddings =  (1, 246, 256)
Shape of generated embeddings =  (1, 313, 256)
Shape of generated embeddings =  (1, 98, 256)
Shape of generated embeddings =  (1, 322, 256)
Shape of generated embeddings =  (1, 620, 256)
Shape of generated embeddings =  (1, 175, 256)
Shape of generated embeddings =  (1, 235, 256)
Shape of generated embeddings =  (1, 134, 256)
Shape of generated embeddings =  (1, 231, 256)
Shape of generated embeddings =  (1, 128, 256)
Shape of generated embeddings =  (1, 195, 256)
Shape of generated embeddings =  (1, 100, 256)
Shape of generated embeddings =  (1, 751, 256)
Shape of generated embeddings =  (1, 133, 256)
Shape of generated embeddings =  (1, 138, 256)
Shape of gener

In [18]:
except_lst

[]

In [16]:
#메모리 오류로 임베딩 되지 않는 문서의 인덱스를 찾기위한 리스트
except_test_lst=[]

In [17]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers

for i in range(len(x_test)):
    try:
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            context_ids = batcher.batch_sentences([x_test[i].split()])
            elmo_context_input_ = sess.run(elmo_context_input['weighted_op'],feed_dict={context_character_ids: context_ids}) 
        print("Shape of generated embeddings = ",elmo_context_input_.shape)
        #임베딩된 문서 임베딩 행렬을 저장
        np.save('C:/Users/user/Desktop/Multimodal TextCuboid/IMDB 분류/elmo_embedding/test(IMDB256)/test%d'%i,elmo_context_input_[0])
    except:
        except_test_lst.append(i)

Shape of generated embeddings =  (1, 137, 256)
Shape of generated embeddings =  (1, 120, 256)
Shape of generated embeddings =  (1, 187, 256)
Shape of generated embeddings =  (1, 240, 256)
Shape of generated embeddings =  (1, 52, 256)
Shape of generated embeddings =  (1, 153, 256)
Shape of generated embeddings =  (1, 128, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 78, 256)
Shape of generated embeddings =  (1, 178, 256)
Shape of generated embeddings =  (1, 243, 256)
Shape of generated embeddings =  (1, 131, 256)
Shape of generated embeddings =  (1, 301, 256)
Shape of generated embeddings =  (1, 311, 256)
Shape of generated embeddings =  (1, 164, 256)
Shape of generated embeddings =  (1, 59, 256)
Shape of generated embeddings =  (1, 359, 256)
Shape of generated embeddings =  (1, 136, 256)
Shape of generated embeddings =  (1, 177, 256)
Shape of generated embeddings =  (1, 131, 256)
Shape of generated embeddings =  (1, 120, 256)
Shape of generate

Shape of generated embeddings =  (1, 586, 256)
Shape of generated embeddings =  (1, 156, 256)
Shape of generated embeddings =  (1, 51, 256)
Shape of generated embeddings =  (1, 550, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 116, 256)
Shape of generated embeddings =  (1, 553, 256)
Shape of generated embeddings =  (1, 106, 256)
Shape of generated embeddings =  (1, 765, 256)
Shape of generated embeddings =  (1, 495, 256)
Shape of generated embeddings =  (1, 177, 256)
Shape of generated embeddings =  (1, 144, 256)
Shape of generated embeddings =  (1, 101, 256)
Shape of generated embeddings =  (1, 177, 256)
Shape of generated embeddings =  (1, 437, 256)
Shape of generated embeddings =  (1, 313, 256)
Shape of generated embeddings =  (1, 358, 256)
Shape of generated embeddings =  (1, 700, 256)
Shape of generated embeddings =  (1, 146, 256)
Shape of generated embeddings =  (1, 73, 256)
Shape of generated embeddings =  (1, 113, 256)
Shape of generat

Shape of generated embeddings =  (1, 143, 256)
Shape of generated embeddings =  (1, 459, 256)
Shape of generated embeddings =  (1, 152, 256)
Shape of generated embeddings =  (1, 826, 256)
Shape of generated embeddings =  (1, 268, 256)
Shape of generated embeddings =  (1, 219, 256)
Shape of generated embeddings =  (1, 287, 256)
Shape of generated embeddings =  (1, 87, 256)
Shape of generated embeddings =  (1, 161, 256)
Shape of generated embeddings =  (1, 162, 256)
Shape of generated embeddings =  (1, 157, 256)
Shape of generated embeddings =  (1, 155, 256)
Shape of generated embeddings =  (1, 218, 256)
Shape of generated embeddings =  (1, 200, 256)
Shape of generated embeddings =  (1, 25, 256)
Shape of generated embeddings =  (1, 153, 256)
Shape of generated embeddings =  (1, 563, 256)
Shape of generated embeddings =  (1, 134, 256)
Shape of generated embeddings =  (1, 357, 256)
Shape of generated embeddings =  (1, 138, 256)
Shape of generated embeddings =  (1, 143, 256)
Shape of genera

Shape of generated embeddings =  (1, 247, 256)
Shape of generated embeddings =  (1, 445, 256)
Shape of generated embeddings =  (1, 98, 256)
Shape of generated embeddings =  (1, 134, 256)
Shape of generated embeddings =  (1, 149, 256)
Shape of generated embeddings =  (1, 215, 256)
Shape of generated embeddings =  (1, 905, 256)
Shape of generated embeddings =  (1, 111, 256)
Shape of generated embeddings =  (1, 259, 256)
Shape of generated embeddings =  (1, 187, 256)
Shape of generated embeddings =  (1, 290, 256)
Shape of generated embeddings =  (1, 229, 256)
Shape of generated embeddings =  (1, 334, 256)
Shape of generated embeddings =  (1, 187, 256)
Shape of generated embeddings =  (1, 686, 256)
Shape of generated embeddings =  (1, 123, 256)
Shape of generated embeddings =  (1, 123, 256)
Shape of generated embeddings =  (1, 653, 256)
Shape of generated embeddings =  (1, 77, 256)
Shape of generated embeddings =  (1, 115, 256)
Shape of generated embeddings =  (1, 304, 256)
Shape of genera

Shape of generated embeddings =  (1, 229, 256)
Shape of generated embeddings =  (1, 190, 256)
Shape of generated embeddings =  (1, 256, 256)
Shape of generated embeddings =  (1, 217, 256)
Shape of generated embeddings =  (1, 227, 256)
Shape of generated embeddings =  (1, 250, 256)
Shape of generated embeddings =  (1, 189, 256)
Shape of generated embeddings =  (1, 149, 256)
Shape of generated embeddings =  (1, 153, 256)
Shape of generated embeddings =  (1, 221, 256)
Shape of generated embeddings =  (1, 264, 256)
Shape of generated embeddings =  (1, 602, 256)
Shape of generated embeddings =  (1, 129, 256)
Shape of generated embeddings =  (1, 121, 256)
Shape of generated embeddings =  (1, 165, 256)
Shape of generated embeddings =  (1, 228, 256)
Shape of generated embeddings =  (1, 127, 256)
Shape of generated embeddings =  (1, 737, 256)
Shape of generated embeddings =  (1, 269, 256)
Shape of generated embeddings =  (1, 123, 256)
Shape of generated embeddings =  (1, 269, 256)
Shape of gene

...

Shape of generated embeddings =  (1, 585, 256)
Shape of generated embeddings =  (1, 667, 256)
Shape of generated embeddings =  (1, 115, 256)
Shape of generated embeddings =  (1, 459, 256)
Shape of generated embeddings =  (1, 106, 256)
Shape of generated embeddings =  (1, 65, 256)
Shape of generated embeddings =  (1, 126, 256)
Shape of generated embeddings =  (1, 149, 256)
Shape of generated embeddings =  (1, 368, 256)
Shape of generated embeddings =  (1, 89, 256)
Shape of generated embeddings =  (1, 112, 256)
Shape of generated embeddings =  (1, 330, 256)
Shape of generated embeddings =  (1, 103, 256)
Shape of generated embeddings =  (1, 199, 256)
Shape of generated embeddings =  (1, 138, 256)
Shape of generated embeddings =  (1, 100, 256)
Shape of generated embeddings =  (1, 272, 256)
Shape of generated embeddings =  (1, 141, 256)
Shape of generated embeddings =  (1, 154, 256)
Shape of generated embeddings =  (1, 228, 256)
Shape of generated embeddings =  (1, 63, 256)
Shape of generat

Shape of generated embeddings =  (1, 276, 256)
Shape of generated embeddings =  (1, 153, 256)
Shape of generated embeddings =  (1, 192, 256)
Shape of generated embeddings =  (1, 69, 256)
Shape of generated embeddings =  (1, 200, 256)
Shape of generated embeddings =  (1, 125, 256)
Shape of generated embeddings =  (1, 560, 256)
Shape of generated embeddings =  (1, 105, 256)
Shape of generated embeddings =  (1, 85, 256)
Shape of generated embeddings =  (1, 120, 256)
Shape of generated embeddings =  (1, 401, 256)
Shape of generated embeddings =  (1, 169, 256)
Shape of generated embeddings =  (1, 147, 256)
Shape of generated embeddings =  (1, 460, 256)
Shape of generated embeddings =  (1, 417, 256)
Shape of generated embeddings =  (1, 117, 256)
Shape of generated embeddings =  (1, 112, 256)
Shape of generated embeddings =  (1, 485, 256)
Shape of generated embeddings =  (1, 134, 256)
Shape of generated embeddings =  (1, 93, 256)
Shape of generated embeddings =  (1, 128, 256)
Shape of generat

Shape of generated embeddings =  (1, 126, 256)
Shape of generated embeddings =  (1, 210, 256)
Shape of generated embeddings =  (1, 130, 256)
Shape of generated embeddings =  (1, 139, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 88, 256)
Shape of generated embeddings =  (1, 147, 256)
Shape of generated embeddings =  (1, 213, 256)
Shape of generated embeddings =  (1, 213, 256)
Shape of generated embeddings =  (1, 80, 256)
Shape of generated embeddings =  (1, 964, 256)
Shape of generated embeddings =  (1, 715, 256)
Shape of generated embeddings =  (1, 140, 256)
Shape of generated embeddings =  (1, 82, 256)
Shape of generated embeddings =  (1, 381, 256)
Shape of generated embeddings =  (1, 66, 256)
Shape of generated embeddings =  (1, 223, 256)
Shape of generated embeddings =  (1, 500, 256)
Shape of generated embeddings =  (1, 101, 256)
Shape of generated embeddings =  (1, 209, 256)
Shape of generated embeddings =  (1, 75, 256)
Shape of generated 

Shape of generated embeddings =  (1, 103, 256)
Shape of generated embeddings =  (1, 988, 256)
Shape of generated embeddings =  (1, 109, 256)
Shape of generated embeddings =  (1, 287, 256)
Shape of generated embeddings =  (1, 160, 256)
Shape of generated embeddings =  (1, 205, 256)
Shape of generated embeddings =  (1, 168, 256)
Shape of generated embeddings =  (1, 172, 256)
Shape of generated embeddings =  (1, 222, 256)
Shape of generated embeddings =  (1, 128, 256)
Shape of generated embeddings =  (1, 174, 256)
Shape of generated embeddings =  (1, 128, 256)
Shape of generated embeddings =  (1, 62, 256)
Shape of generated embeddings =  (1, 128, 256)
Shape of generated embeddings =  (1, 118, 256)
Shape of generated embeddings =  (1, 246, 256)
Shape of generated embeddings =  (1, 177, 256)
Shape of generated embeddings =  (1, 131, 256)
Shape of generated embeddings =  (1, 123, 256)
Shape of generated embeddings =  (1, 175, 256)
Shape of generated embeddings =  (1, 356, 256)
Shape of gener

Shape of generated embeddings =  (1, 79, 256)
Shape of generated embeddings =  (1, 342, 256)
Shape of generated embeddings =  (1, 121, 256)
Shape of generated embeddings =  (1, 76, 256)
Shape of generated embeddings =  (1, 112, 256)
Shape of generated embeddings =  (1, 319, 256)
Shape of generated embeddings =  (1, 67, 256)
Shape of generated embeddings =  (1, 137, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 299, 256)
Shape of generated embeddings =  (1, 303, 256)
Shape of generated embeddings =  (1, 440, 256)
Shape of generated embeddings =  (1, 249, 256)
Shape of generated embeddings =  (1, 110, 256)
Shape of generated embeddings =  (1, 219, 256)
Shape of generated embeddings =  (1, 131, 256)
Shape of generated embeddings =  (1, 284, 256)
Shape of generated embeddings =  (1, 308, 256)
Shape of generated embeddings =  (1, 177, 256)
Shape of generated embeddings =  (1, 79, 256)
Shape of generated embeddings =  (1, 267, 256)
Shape of generated

In [18]:
except_test_lst

[]